# Database 2
## Last lecture: SQL query clauses
- FROM: table
- SELECT: columns
- WHERE: row condition -> boolean expression (recommended to do before LIMIT)
    - boolean operators: AND, OR, NOT
    - AND / OR: can be used to combine conditions
- LIMIT: simple limiation of number of rows
- GROUP BY: sorting

## Today's lecture:
- aggregation: SUM, AVG, COUNT, MIN, MAX
- group by: equivalent to bucketization; one row can only be part of one bucket
- having: applying condition to groups

In [1]:
# ignore this cell (it's just to make certain text red later, but you don't need to understand it).
from IPython.core.display import HTML
HTML('<style>em { color: red; }</style>')

In [2]:
# import statements
import sqlite3
import pandas as pd
import os

In [3]:
movies_path = "movies.db"
assert os.path.exists(movies_path)

c = sqlite3.connect(movies_path)
c

In [4]:
pd.read_sql("select * from sqlite_master", c)

,type,name,tbl_name,rootpage,sql
0,table,movies,movies,2,"CREATE TABLE ""movies"" (\n""Title"" TEXT,\n ""Dir..."


In [5]:
pd.read_sql("select * from movies", c).head(5)

,Title,Director,Year,Runtime,Rating,Revenue
0,Guardians of the Galaxy,James Gunn,2014,121,8.1,333.13
1,Prometheus,Ridley Scott,2012,124,7.0,126.46
2,Split,M. Night Shyamalan,2016,117,7.3,138.12
3,Sing,Christophe Lourdelet,2016,108,7.2,270.32
4,Suicide Squad,David Ayer,2016,123,6.2,325.02


In [6]:
def qry(sql, conn = c):
    return pd.read_sql(sql, conn)

In [7]:
qry("""
SELECT *
FROM movies
""")

,Title,Director,Year,Runtime,Rating,Revenue
0,Guardians of the Galaxy,James Gunn,2014,121,8.1,333.13
1,Prometheus,Ridley Scott,2012,124,7.0,126.46
2,Split,M. Night Shyamalan,2016,117,7.3,138.12
3,Sing,Christophe Lourdelet,2016,108,7.2,270.32
4,Suicide Squad,David Ayer,2016,123,6.2,325.02
...,...,...,...,...,...,...
993,Secret in Their Eyes,Billy Ray,2015,111,6.2,0.00
994,Hostel: Part II,Eli Roth,2007,94,5.5,17.54
995,Step Up 2: The Streets,Jon M. Chu,2008,98,6.2,58.01
996,Search Party,Scot Armstrong,2014,93,5.6,0.00


# Review: Simple Selections

### Which *movie* has the *highest rating*?

In [8]:
qry("""
SELECT Title, rating
FROM movies
ORDER BY Rating DESC
LIMIT 1
""")

,Title,Rating
0,The Dark Knight,9.0


### Which *director* made the *shortest movie*?

In [9]:
qry("""
SELECT director, runtime
FROM movies
ORDER BY runtime
LIMIT 1
""")

,Director,Runtime
0,Claude Barras,66


### Which *director* made the *highest-revenue movie*?

In [10]:
qry("""
SELECT director, revenue, title
FROM movies
ORDER BY revenue DESC
LIMIT 1
""")

,Director,Revenue,Title
0,J.J. Abrams,936.63,Star Wars: Episode VII - The Force Awakens


### Which *movie* had the *highest revenues* in *2016*?

In [11]:
qry("""
SELECT title, revenue, year
FROM movies
WHERE year = 2016
ORDER BY revenue DESC
LIMIT 1
""")

,Title,Revenue,Year
0,Rogue One,532.17,2016


### Which *3 movies* had the *highest revenues* in *2016*?

In [12]:
qry("""
SELECT title, revenue
FROM movies
WHERE year = 2016
ORDER BY revenue DESC
LIMIT 3
""")

,Title,Revenue
0,Rogue One,532.17
1,Finding Dory,486.29
2,Captain America: Civil War,408.08


### Which *3 movies* have the *highest rating-to-revenue ratios*?

Introduce `AS`

In [13]:
qry("""
SELECT title, rating / revenue AS ratio
FROM movies
ORDER BY ratio DESC
LIMIT 3
""")

,Title,ratio
0,Wakefield,750.0
1,"Love, Rosie",720.0
2,Lovesong,640.0


# Aggregate Queries

```
SUM, AVG, COUNT, MIN, MAX
```

### How many *movies* are there?

In [14]:
qry("""
SELECT COUNT(*)
FROM movies
""")

,COUNT(*)
0,998


### How many *directors* are there?

In [15]:
# This doesn't feel correct - it counts duplicates for director names!
qry("""
SELECT COUNT(director)
FROM movies
""")

,COUNT(director)
0,998


In [16]:
qry("""
SELECT COUNT(DISTINCT director)
FROM movies
""")

,COUNT(DISTINCT director)
0,643


### What is the *total revenue* of *all the movies*?

In [17]:
qry("""
SELECT SUM(revenue)
FROM movies
""")

,SUM(revenue)
0,72215.45


### What is the *average rating* across *all movies*?

* v1: with `SUM` and `COUNT`
* v2: with `AVG`

In [18]:
# v1
qry("""
SELECT SUM(rating) / COUNT(*)
FROM movies
""")

,SUM(rating) / COUNT(*)
0,6.723447


In [19]:
# v2
qry("""
SELECT AVG(rating)
FROM movies
""")

,AVG(rating)
0,6.723447


### What is the *average revenue* and *average runtime* of *all the movies*?

In [20]:
qry("""
SELECT AVG(revenue), AVG(runtime)
FROM movies
""")

,AVG(revenue),AVG(runtime)
0,72.36017,113.170341


### What is the *average runtime* for a *James Gunn* movie?

In [21]:
qry("""
SELECT AVG(runtime)
FROM movies
WHERE director = "James Gunn"
""")

,AVG(runtime)
0,104.0


### What is the *average revenue* for a *Ridley Scott* movie?

In [22]:
qry("""
SELECT AVG(revenue)
FROM movies
WHERE director = "Ridley Scott"
""")

,AVG(revenue)
0,89.8825


### *How many movies* were there in *2016*?

In [23]:
qry("""
SELECT COUNT(*)
FROM movies
WHERE year = 2016
""")

,COUNT(*)
0,296


### What *percentage* of the *total revenue* came from the *highest-revenue movie*?

In [24]:
qry("""
SELECT MAX(revenue) / SUM(revenue) * 100 AS percentage
FROM movies
""")

,percentage
0,1.296994


### What *percentage* of the *revenue* came from the *highest-revenue movie* in *2016*?

In [25]:
qry("""
SELECT MAX(revenue) / SUM(revenue) * 100 AS percentage
FROM movies
WHERE year = 2016
""")

,percentage
0,4.746581


### Follow up question: *which movie* was this?

In [26]:
qry("""
SELECT *
FROM movies
WHERE year = 2016
ORDER BY revenue DESC
LIMIT 1
""")

,Title,Director,Year,Runtime,Rating,Revenue
0,Rogue One,Gareth Edwards,2016,133,7.9,532.17


# GROUP BY Queries

```sql
SELECT ???, ??? FROM Movies
GROUP BY ???
```

### What is the *total revenue* per each *year*?

* v1: the amounts
* v2: the amounts, as labeled by year

In [27]:
# v1
qry("""
SELECT SUM(revenue)
FROM movies
GROUP BY year
""")

,SUM(revenue)
0,3624.46
1,4306.23
2,5053.22
3,5292.26
4,5989.65
5,5431.96
6,6910.29
7,7544.21
8,7997.40
9,8854.12


In [28]:
# v2
qry("""
SELECT year, SUM(revenue)
FROM movies
GROUP BY year
""")

,Year,SUM(revenue)
0,2006,3624.46
1,2007,4306.23
2,2008,5053.22
3,2009,5292.26
4,2010,5989.65
5,2011,5431.96
6,2012,6910.29
7,2013,7544.21
8,2014,7997.40
9,2015,8854.12


### *How many movies* were by each *director*?

In [29]:
qry("""
SELECT director, COUNT(*) AS mov_count
FROM movies
GROUP BY director
ORDER BY mov_count DESC
""")

,Director,mov_count
0,Ridley Scott,8
1,Paul W.S. Anderson,6
2,Michael Bay,6
3,M. Night Shyamalan,6
4,David Yates,6
...,...,...
638,Aisling Walsh,1
639,Afonso Poyart,1
640,Adam Leon,1
641,Abdellatif Kechiche,1


### What is the *average rating* for each *director*?

In [30]:
qry("""
SELECT director, AVG(rating)
FROM movies
GROUP BY director
""")

,Director,AVG(rating)
0,Aamir Khan,8.50
1,Abdellatif Kechiche,7.80
2,Adam Leon,6.50
3,Adam McKay,7.00
4,Adam Shankman,6.30
...,...,...
638,Xavier Dolan,7.55
639,Yimou Zhang,6.10
640,Yorgos Lanthimos,7.20
641,Zack Snyder,7.04


### What is the *average runtime* for each *director*?

In [31]:
qry("""
SELECT director, AVG(runtime)
FROM movies
GROUP BY director
""")

,Director,AVG(runtime)
0,Aamir Khan,165.00
1,Abdellatif Kechiche,180.00
2,Adam Leon,82.00
3,Adam McKay,110.75
4,Adam Shankman,120.00
...,...,...
638,Xavier Dolan,118.00
639,Yimou Zhang,103.00
640,Yorgos Lanthimos,106.50
641,Zack Snyder,136.60


### How many *unique directors* created a movie in each *year*

In [32]:
qry("""
SELECT year, COUNT(DISTINCT director) AS director_count
FROM movies
GROUP BY year
""")

,Year,director_count
0,2006,44
1,2007,51
2,2008,51
3,2009,51
4,2010,60
5,2011,63
6,2012,64
7,2013,88
8,2014,97
9,2015,127


# Combining GROUP BY with other CLAUSES

<img src="groupby.png">

### What is the *total revenue* of per *year*, in *recent* years?

In [33]:
# recent means 5 years
qry("""
SELECT year, SUM(revenue) AS total_revenue
FROM movies
GROUP BY Year
ORDER BY Year DESC
LIMIT 5
""")

,Year,total_revenue
0,2016,11211.65
1,2015,8854.12
2,2014,7997.40
3,2013,7544.21
4,2012,6910.29


### Which *directors* have had the *largest number of movies* earning *over 100M dollars*?

In [34]:
qry("""
SELECT director, COUNT(title) AS count
FROM movies
WHERE revenue > 100
GROUP BY director
ORDER BY count DESC
""")

,Director,count
0,David Yates,6
1,J.J. Abrams,5
2,Zack Snyder,4
3,Ridley Scott,4
4,Paul Feig,4
...,...,...
156,Alfonso Cuarón,1
157,Alessandro Carloni,1
158,Alejandro González Iñárritu,1
159,Alan Taylor,1


### Which *three* of the *directors* have the *greatest average rating*?

In [35]:
qry("""
SELECT director, AVG(rating) AS avg_rating, COUNT(*) as count
FROM movies
GROUP BY director
ORDER BY avg_rating DESC
LIMIT 3
""")

,Director,avg_rating,count
0,Nitesh Tiwari,8.80,1
1,Christopher Nolan,8.68,5
2,Olivier Nakache,8.60,1


Why is the above question maybe not the best to ask?

In [36]:
# We want to consider if the director has multiple great movies, instead of just one

### Which *three* of the *directors* have the *greatest average rating* over at *least three movies*?

In [37]:
# We cannot use where clause on aggregates because that data doesn't exist in the original table
# qry("""
# SELECT director, AVG(rating) AS avg_rating, COUNT(*) as count
# FROM movies
# WHERE count >= 3
# GROUP BY director
# ORDER BY avg_rating DESC
# LIMIT 3
# """)

Need filtering BEFORE and AFTER the GROUP operations

<img src="pipeline.png">

# WHERE vs. HAVING

* WHERE: filter rows in original table
* HAVING: filter groups

### Repeat: Which *three* of the *directors* have the *greatest average rating* over at *least three movies*?

<img src="having.png">

In [38]:
# We cannot use where clause on aggregates because that data doesn't exist in the original table
qry("""
SELECT director, AVG(rating) AS avg_rating, COUNT(*) as count
FROM movies
GROUP BY director
HAVING count >= 3
ORDER BY avg_rating DESC
LIMIT 3
""")

,Director,avg_rating,count
0,Christopher Nolan,8.68,5
1,Martin Scorsese,7.92,5
2,Quentin Tarantino,7.90,4


### Which *directors* have had *more than 3 movies* that have been *since 2010*?

In [39]:
qry("""
SELECT director, COUNT(title) AS count
FROM movies
WHERE year >= 2010
GROUP BY director
HAVING count > 3
""")

,Director,count
0,Antoine Fuqua,4
1,David O. Russell,4
2,David Yates,4
3,Denis Villeneuve,5
4,James Wan,4
5,M. Night Shyamalan,4
6,Martin Scorsese,4
7,Michael Bay,4
8,Mike Flanagan,4
9,Paul Feig,4


### Which *directors* have had more than *three* movies with runtimes under *100* minutes

In [40]:
qry("""
SELECT director, COUNT(title) AS count
FROM movies
WHERE runtime < 100
GROUP BY director
HAVING count > 3
""")

,Director,count
0,Woody Allen,4


In [41]:
# Don't forget to close the movies.db connection
c.close()

# Practice: Survey data

In [42]:
# open a connection to survey.db
survey_path = 'survey.db'
assert os.path.exists(survey_path)

conn = sqlite3.connect(survey_path)
conn

In [43]:
qry("""
select *
from sqlite_master
""", conn)

,type,name,tbl_name,rootpage,sql
0,table,fall_2021,fall_2021,2,"CREATE TABLE ""fall_2021"" (\n""index"" INTEGER,\n..."
1,index,ix_fall_2021_index,fall_2021,3,"CREATE INDEX ""ix_fall_2021_index""ON ""fall_2021..."


### Take a peek at fall_2021 table data

In [44]:
qry("""
select *
from fall_2021
""", conn)

,index,lecture,age,major,topping
0,0,LEC001,19.0,Computer Science,basil/spinach
1,1,LEC002,18.0,Engineering,pineapple
2,2,LEC003,19.0,Business,pepperoni
3,3,LEC003,19.0,Engineering,Other
4,4,LEC001,19.0,Data Science,sausage
...,...,...,...,...,...
891,891,LEC001,19.0,Data Science,pepperoni
892,892,LEC001,20.0,Data Science,pepperoni
893,893,LEC004,21.0,Computer Science,pepperoni
894,894,LEC001,19.0,Engineering,tomato


### How many students in *LEC003* are graduating with *Engineering* major?

In [45]:
qry("""
select COUNT(*)
from fall_2021
where lecture = "LEC003" AND major = "Engineering"
""", conn)

,COUNT(*)
0,88


### How many students are in *each major*?
- bonus: sort based on majors, with most popular at the top

In [46]:
qry("""
select major, count(*) as major_count
from fall_2021
group by major
order by major_count DESC
""", conn)

,major,major_count
0,Engineering,328
1,Data Science,178
2,Computer Science,151
3,Business,93
4,Natural Science,52
5,Social Science,24
6,Statistics,6
7,Economics,5
8,undecided,4
9,Mathematics,4


### What are the *top 5 popular majors*?

In [47]:
qry("""
select major, count(*) as major_count
from fall_2021
group by major
order by major_count DESC
LIMIT 5
""", conn)

,major,major_count
0,Engineering,328
1,Data Science,178
2,Computer Science,151
3,Business,93
4,Natural Science,52


### What is the *average age* for *each major* with *at least 10 people*?
- bonus: sort based on popular major

In [48]:
qry("""
select major, count(*) as major_count, AVG(age) as average_age
from fall_2021
group by major
having major_count > 10
order by major_count DESC
""", conn)

,major,major_count,average_age
0,Engineering,328,19.418462
1,Data Science,178,19.811429
2,Computer Science,151,19.693878
3,Business,93,19.673913
4,Natural Science,52,21.078431
5,Social Science,24,19.695652


### How many *CS or DS majors* are in *each lecture*?

In [49]:
qry("""
select lecture, count(*) as lecture_count
from fall_2021
where major = "Computer Science" OR major = "Data Science"
group by lecture
""", conn)

,lecture,lecture_count
0,LEC001,108
1,LEC002,58
2,LEC003,49
3,LEC004,62
4,LEC005,52


### What are the *top 10 pizza toppings*?

In [50]:
qry("""
select topping, count(*) as topping_count
from fall_2021
group by topping
order by topping_count DESC
LIMIT 10
""", conn)

,topping,topping_count
0,pepperoni,261
1,sausage,190
2,pineapple,93
3,mushroom,78
4,none (just cheese),75
5,basil/spinach,60
6,Other,57
7,macaroni/pasta,40
8,tomato,26
9,green pepper,16


### Which 2 lectures like pineapple the most?

In [51]:
qry("""
select lecture, count(*) as lecture_count
from fall_2021
where topping = "pineapple"
group by lecture
order by lecture_count DESC
LIMIT 2
""", conn)

,lecture,lecture_count
0,LEC001,29
1,LEC004,20


In [52]:
c.close()